# Test PointPillars on ARCS data

In [199]:
# Libraries
import bbox
import itertools
import math
import os
import random
import sys
import time
import numpy as np
import pandas as pd
from contextlib import redirect_stdout
from mmdet3d.apis import LidarDet3DInferencer
from pathlib import Path

In [200]:
# Initialize inferencer
inferencer = LidarDet3DInferencer('pointpillars_kitti-3class')

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmdetection3d/v1.0.0_models/pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class_20220301_150306-37dc2420.pth


/home/rachel/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(
/home/rachel/miniconda3/envs/openmm_mmvc/lib/python3.8/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [201]:
# Directory paths
ARCS_LABELS = '../data/eval_data/labels'
ARCS_DATA_DIR = '../data/eval_data/arcs'
PREV_ARCS_AZIMUTH_FILTERED_DATA_DIR = '../data/eval_data/arcs_azimuth_filtered_01'
ARCS_AZIMUTH_FILTERED_DATA_DIR = '../data/eval_data/arcs_azimuth_filtered'
ARCS_LABEL_FILTERED_DATA_DIR = '../data/eval_data/arcs_label_filtered'

## Evaluation functions

In [202]:
# The 3D bbox object requires quaternion values. This function converts the yaw to these values
def yaw_to_quaternion(yaw):
    """
    Converts a yaw angle (rotation about the z-axis) to quaternion coordinates.
    
    Parameters:
    - yaw (float): The yaw angle in radians.
    
    Returns:
    - tuple of (rw, rx, ry, rz): The quaternion representation of the yaw.
    """
    # Compute the components of the quaternion
    rw = math.cos(yaw / 2.0)
    rz = math.sin(yaw / 2.0)
    
    # rx and ry are zero because the rotation is only about the z-axis
    return (rw, 0, 0, rz)

In [203]:
# This function takes x, y, z, dx, dy, dz, yaw and returns a 3D bbox object from the bbox package
# Yaw is in radians
def get3DBbox(x, y, z, dx, dy, dz, yaw):
    # Example usage:
    rw, rx, ry, rz = yaw_to_quaternion(yaw)
    bbox_obj = bbox.BBox3D(x, y, z, length=dx, width=dy, height=dz, rw=rw, rx=rx, ry=ry, rz=rz)
    return bbox_obj

In [204]:
# Takes two bounding boxes, returns the IoU
def calculate_iou(bbox1, bbox2):
    bbox_obj1 = get3DBbox(*bbox1)
    bbox_obj2 = get3DBbox(*bbox2)
    return bbox.metrics.jaccard_index_3d(bbox_obj1, bbox_obj2)

In [205]:
evaluation_area = [21.0175, 11.717, -0.3925, 40.035, 55.9349, 7.535, 0.5]

In [206]:
# Returns true if the pending box overlaps the evaluation area
def is_in_eval_area(pending_bbox):
    iou = calculate_iou(evaluation_area, pending_bbox)
    if iou > 0:
        return True
    return False
#     return True

In [207]:
# _evaluate frame takes a prediction dictionary output by the model, and a list of ground truths
# from the label file, and returns the TPs, FPs, and FNs for one lidar frame
def _evaluate_frame(predictions, ground_truths, iou_threshold=0.23):
    TPs, FPs, FNs = 0, 0, len(ground_truths)
    used_gt = set()
    confidence_labels = []
    
    for pred in predictions['predictions']:
        for label, score, bbox in zip(pred['labels_3d'], pred['scores_3d'], pred['bboxes_3d']):
            # Only count the prediction if it's within the evaluation area
            if is_in_eval_area(bbox):
                best_iou = 0
                best_gt = None
                for gt in ground_truths:
                    iou = calculate_iou(bbox, gt[8:])
                    if iou > best_iou:
                        best_iou = iou
                        best_gt = tuple(gt)

                if best_iou > iou_threshold:
                    if best_gt not in used_gt:
                        used_gt.add(best_gt)
                        TPs += 1
                        FNs -= 1
                        confidence_labels.append((score, 1))
                else:
                    FPs += 1
                    confidence_labels.append((score, 0))
#                 else:
#                     FPs += 1
    return TPs, FPs, FNs, confidence_labels

In [208]:
def parse_ground_truths(label_path):
    labels = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            bbox = []
            # Add the category as a string
            bbox.append(parts[0])
            # Extract the bounding box dimensions and location as 
            bbox = bbox + [float(value) for value in parts[1:15]]  
            bbox[2] = int(bbox[2])
            # Only include the bbox if it's in the evaluation area
            if is_in_eval_area(bbox[8:]):
                labels.append(bbox)
    return labels

In [209]:
generated_problem_file_list = []

In [210]:
# _evaluate frame takes a lidar and label file path and
# and returns the TPs, FPs, and FNs for one lidar frame
def evaluate_frame(lidar_file_path, label_file_path):
    inputs = dict(points=str(lidar_file_path))
    
    # Get predictions
    try:
        predictions = inferencer(inputs)
            # Get ground truths
        ground_truths = parse_ground_truths(label_file_path)
        num_ground_truths = len(ground_truths)

        TPs, FPs, FNs, confidence_labels = _evaluate_frame(predictions, ground_truths)
    except:
        # Add file id to the problem file list
        print(str(lidar_file_path))
        lidar_filename = os.path.basename(lidar_file_path)
        file_id, extension = os.path.splitext(lidar_filename)
        generated_problem_file_list.append(file_id)
        TPs, FPs, FNs, num_ground_truths, confidence_labels = 0, 0, 0, 0, []
    
    return TPs, FPs, FNs, num_ground_truths, confidence_labels

In [211]:
# Problem files. I'll find out why later
problem_file_ids = ['000522', '003066', '000556', '004340', '001016', '004224', '005318', '004344', '001014', '000522',
                    '003066', '000556', '004340', '001016', '004224', '005318', '004344', '001014', '001027', '006440',
                    '001026', '000869', '001031', '000530', '004928', '000548', '000553', '004290', '006437', '000548',
                    '000553', '004290', '006437', '000542', '001015', '006443', '005320', '000542', '001015', '006443', 
                    '005320', '004015', '003064', '000521', '000536', '003144', '001019', '000559', '000524', '005158', 
                    '005155', '000540', '004291', '000444', '006444', '004225', '000545', '004343', '000665', '003395', 
                    '003439', '001018', '000531', '000534', '004013', '004054', '001020', '004250', '006441', '000863', 
                    '000767']

In [212]:
def print_summary(list_file_ids):
    for id in list_file_ids:
        print(f'\n_____________________________________________________________')
        
        file_path = f'{ARCS_LABEL_FILTERED_DATA_DIR}/{id}.bin'
        label_path = f'{ARCS_LABELS}/{id}.txt' 

        # Open point cloud file
        points = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
        df_points = pd.DataFrame(points, columns=['x', 'y', 'z', 'intensity'])

        # Print the DataFrame summary
        print(f"Summary for file ID {id}:")
        print('length: ' + str(len(df_points)))
        print("DataFrame Info:")
        df_points.info()
        print("\nDataFrame Description:")
        print(df_points.describe())
        print(f'\nlabels:')
        # Print labels
        with open(label_path, 'r') as file:
            for line in file:
                print(line)

## Metrics calculation functions

In [213]:
def calculate_precision_recall(predictions, num_ground_truths):
    # Sort by confidence score in descending order
    predictions.sort(key=lambda x: x[0], reverse=True)
    
    tp = 0
    fp = 0
    total_positives = sum(is_tp for _, is_tp in predictions)
    
    precisions = []
    recalls = []
    
    for conf, is_tp in predictions:
        if is_tp:
            tp += 1
        else:
            fp += 1
        
        precision = tp / (tp + fp)
        recall = tp / num_ground_truths
        
        precisions.append(precision)
        recalls.append(recall)
    
    return precisions, recalls

In [214]:
def calculate_ap(precisions, recalls):
    # Calculate AP using the trapezoidal rule to compute the area under the curve
    ap = 0
    for i in range(1, len(recalls)):
        ap += (recalls[i] - recalls[i-1]) * (precisions[i] + precisions[i-1]) / 2
    return ap

In [215]:
test_dir = Path(ARCS_DATA_DIR)
files = [f for f in os.listdir(test_dir) if f.endswith('.bin')]
random.shuffle(files)
files = files[:200]
files = [file for file in files if file[:-4] not in problem_file_ids]
print(files)

['006074.bin', '001362.bin', '002898.bin', '001297.bin', '002870.bin', '004305.bin', '005765.bin', '002463.bin', '005494.bin', '003235.bin', '005264.bin', '005633.bin', '004068.bin', '004681.bin', '006184.bin', '005405.bin', '002731.bin', '001869.bin', '004017.bin', '001102.bin', '002723.bin', '003421.bin', '001943.bin', '002345.bin', '002693.bin', '005768.bin', '005723.bin', '002495.bin', '003200.bin', '002128.bin', '002206.bin', '004012.bin', '006358.bin', '003724.bin', '006066.bin', '001011.bin', '001674.bin', '001551.bin', '003176.bin', '000266.bin', '001845.bin', '002081.bin', '001076.bin', '005937.bin', '002842.bin', '005238.bin', '006191.bin', '004120.bin', '001651.bin', '001384.bin', '004484.bin', '000224.bin', '004424.bin', '002114.bin', '002845.bin', '001588.bin', '003259.bin', '000148.bin', '000082.bin', '001834.bin', '003704.bin', '002779.bin', '000127.bin', '005484.bin', '002047.bin', '005963.bin', '001549.bin', '000326.bin', '005836.bin', '000351.bin', '004653.bin', '0037

In [216]:
def evaluate_dataset(dataset_path, num_frames):
    label_dir = Path(ARCS_LABELS)
    dataset_dir = Path(dataset_path)
    
    total_TPs, total_FPs, total_FNs = 0, 0, 0
    num_ground_truths = 0
    confidence_labels = []

    start = time.time()
    for bin_file in files:
#     for bin_file in itertools.islice(dataset_dir.iterdir(), num_frames):
#     for bin_file in dataset_dir.iterdir():
        if str(bin_file).endswith('.bin'):
            print('.', end='')

            lidar_filename = os.path.basename(bin_file)
            # Split the filename from the extension ('006428', '.txt')
            file_id, extension = os.path.splitext(lidar_filename)
            if file_id not in problem_file_ids:
                label_filename = file_id + '.txt'

                # Make file paths
                lidar_file_path = Path(dataset_dir, lidar_filename)
                label_file_path = Path(label_dir, label_filename)
                
#                 print('evaluating: ' + str(lidar_file_path))
                
                TPs, FPs, FNs, num_frame_ground_truths, frame_confidence_labels = evaluate_frame(lidar_file_path, label_file_path)

                total_TPs += TPs
                total_FPs += FPs
                total_FNs += FNs
                num_ground_truths += num_frame_ground_truths
                confidence_labels += frame_confidence_labels
            else:
                print('skipping ' + file_id)
    end = time.time()
        
    return total_TPs, total_FPs, total_FNs, num_ground_truths, confidence_labels, (end - start) / num_frames

In [217]:
def test_pointpillars(dataset_path, dataset_name, num_frames=200):
    print('Evaluating dataset: ' + dataset_path)
    # Get all TP, FP, and FN in the dataset
    TPs, FPs, FNs, num_ground_truths, confidence_labels, time_per_frame = evaluate_dataset(dataset_path, num_frames)
    # Get metrics
    precision = TPs / (TPs + FPs) if TPs + FPs > 0 else 0
    recall = TPs / (TPs + FNs) if TPs + FNs > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Placeholder for average precision calculation
    precisions, recalls = calculate_precision_recall(confidence_labels, num_ground_truths)
    print(confidence_labels)
    print(precisions)
    print(recalls)
    average_precision = calculate_ap(precisions, recalls)

    # Organize results into a dictionary
    results = {
        'Dataset': dataset_name,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1_score,
        'Average Precision': average_precision,
        'time_per_frame': time_per_frame
    }
    print(results)

    return results

In [218]:
%%capture
# Run tests on ARCS
results = test_pointpillars(ARCS_DATA_DIR, 'ARCS')

In [219]:
%%capture
# Run tests on ARCS filtered
azimuth_filter_results = test_pointpillars(PREV_ARCS_AZIMUTH_FILTERED_DATA_DIR, 'Azimuth Filtered ARCS')

In [220]:
%%capture
# Run tests on ARCS filtered
new_filter_results = test_pointpillars(ARCS_AZIMUTH_FILTERED_DATA_DIR, 'New Azimuth Filtered ARCS')

In [221]:
%%capture
# Run tests on ARCS label filtered
label_filter_results = test_pointpillars(ARCS_LABEL_FILTERED_DATA_DIR, 'Label Filtered ARCS')

In [222]:
print(generated_problem_file_list)

['004012', '001011', '000763', '000443', '000446']


## Show results

In [223]:
# print(results)
# print(filter_results)
# print(label_filter_results)

In [224]:
# Create DataFrame
results_df = pd.DataFrame([results, azimuth_filter_results, new_filter_results, label_filter_results])
display(results_df)

,Dataset,Precision,Recall,F1 Score,Average Precision,time_per_frame
0,ARCS,0.582915,0.746781,0.654751,0.643249,0.146426
1,Azimuth Filtered ARCS,0.522003,0.738197,0.611556,0.579220,0.109838
2,New Azimuth Filtered ARCS,0.429875,0.809013,0.561430,0.671618,0.124676
3,Label Filtered ARCS,0.848416,0.809935,0.828729,0.781788,0.102341


In [225]:
print(results_df.to_string(index=False))

                  Dataset  Precision   Recall  F1 Score  Average Precision  time_per_frame
                     ARCS   0.582915 0.746781  0.654751           0.643249        0.146426
    Azimuth Filtered ARCS   0.522003 0.738197  0.611556           0.579220        0.109838
New Azimuth Filtered ARCS   0.429875 0.809013  0.561430           0.671618        0.124676
      Label Filtered ARCS   0.848416 0.809935  0.828729           0.781788        0.102341
